Задание:
1. Выбрать (скачать) самостоятельно датасет из kaggle.com и построить граф с географической привязкой данных
2. Для решения задания использовать методы библиотеки networkx или иные
3. Рассчитаь среднюю степень вершины, средний минимальный путь и диаметр полученного графа
4. Проанализировать и выявить закономерности при их наличии

In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import numpy as np
from itertools import combinations


In [9]:
# Загрузка датасета
df = pd.read_csv('/content/drive/MyDrive/МОСиСА/data/worldcities.csv')

In [10]:
df = df[df['country'] == 'Poland']
df

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
442,Warsaw,Warsaw,52.2300,21.0111,Poland,PL,POL,Mazowieckie,primary,1860281.0,1616024847
944,Kraków,Krakow,50.0614,19.9372,Poland,PL,POL,Małopolskie,admin,804237.0,1616172264
1109,Wrocław,Wroclaw,51.1100,17.0325,Poland,PL,POL,Dolnośląskie,admin,672929.0,1616108520
1113,Łódź,Lodz,51.7769,19.4547,Poland,PL,POL,Łódzkie,admin,670642.0,1616832750
1353,Poznań,Poznan,52.4083,16.9336,Poland,PL,POL,Wielkopolskie,admin,546859.0,1616725607
...,...,...,...,...,...,...,...,...,...,...,...
45843,Kościelisko,Koscielisko,49.2908,19.8883,Poland,PL,POL,Małopolskie,NaN,8661.0,1616942761
46068,Żuromin,Zuromin,53.0667,19.9000,Poland,PL,POL,Mazowieckie,minor,8592.0,1616921543
46167,Kamień Pomorski,Kamien Pomorski,53.9700,14.7725,Poland,PL,POL,Zachodniopomorskie,minor,8564.0,1616000112
46336,Poniatowa,Poniatowa,51.1928,22.0647,Poland,PL,POL,Lubelskie,NaN,8517.0,1616798001


In [11]:
# Ограничиваем количество городов
# df = df.sample(n=100, random_state=42)  # Берем 100 случайных городов

In [18]:

# Создание графа
G = nx.Graph()

# Добавление вершин (городов)
for index, row in df.iterrows():
    G.add_node(row['city'], pos=(row['lat'], row['lng']))

# Генерация всех уникальных пар городов
city_pairs = list(combinations(df.index, 2))

# Добавление ребер (расстояния между городами)
for i, j in city_pairs:
    row1 = df.loc[i]
    row2 = df.loc[j]
    city1 = (row1['lat'], row1['lng'])
    city2 = (row2['lat'], row2['lng'])
    distance = geodesic(city1, city2).km
    if distance < 500:  # Добавляем ребро, если расстояние меньше 500 км
        G.add_edge(row1['city'], row2['city'], weight=distance)


In [19]:
# Визуализация графа
plt.figure(figsize=(12, 8))

# Получение позиций вершин
pos = nx.get_node_attributes(G, 'pos')

# Рисуем ребра
nx.draw_networkx_edges(G, pos, edge_color='gray', width=0.5, alpha=0.7)

# Рисуем вершины
node_degrees = dict(G.degree())
node_colors = [node_degrees[node] for node in G.nodes()]
nx.draw_networkx_nodes(G, pos, node_size=50, node_color=node_colors, cmap=plt.cm.viridis, alpha=0.8)

# Подписи для вершин (городов)
labels = {node: node for node in G.nodes() if node_degrees[node] > 5}  # Подписываем только города с высокой степенью
nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')

# Цветовая шкала для степени вершин
sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=min(node_colors), vmax=max(node_colors)))
sm.set_array([])
plt.colorbar(sm, label='Степень вершины')

# Настройка графика
plt.title("Граф городов с расстоянием менее 500 км", fontsize=14)
plt.xlabel("Долгота", fontsize=12)
plt.ylabel("Широта", fontsize=12)
plt.grid(True, alpha=0.2)
plt.show()

ValueError: Unable to determine Axes to steal space for Colorbar. Either provide the *cax* argument to use as the Axes for the Colorbar, provide the *ax* argument to steal space from it, or add *mappable* to an Axes.

Error in callback <function flush_figures at 0x7b547d694400> (for post_execute):


KeyboardInterrupt: 

In [17]:
# Расчет средней степени вершины
average_degree = sum(dict(G.degree()).values()) / G.number_of_nodes()
print(f"Средняя степень вершины: {average_degree}")


Средняя степень вершины: 436.4989247311828


In [14]:

# Проверка на связность графа
if nx.is_connected(G):
    # Расчет среднего минимального пути
    average_shortest_path = nx.average_shortest_path_length(G)
    print(f"Средний минимальный путь: {average_shortest_path}")

    # Расчет диаметра графа
    diameter = nx.diameter(G)
    print(f"Диаметр графа: {diameter}")
else:
    # Если граф не связный, обрабатываем каждую компоненту связности отдельно
    print("Граф не связный. Обработка компонент связности:")
    components = nx.connected_components(G)
    for i, component in enumerate(components, 1):
        subgraph = G.subgraph(component)
        print(f"\nКомпонента связности {i}:")
        print(f"Количество городов: {len(component)}")
        if len(component) > 1:
            average_shortest_path = nx.average_shortest_path_length(subgraph)
            print(f"Средний минимальный путь: {average_shortest_path}")
            diameter = nx.diameter(subgraph)
            print(f"Диаметр графа: {diameter}")
        else:
            print("Компонента содержит только один город.")


Средний минимальный путь: 1.0592695587690026
Диаметр графа: 2
